In [ ]:
import logging
import asyncio
import polars as pl
import plotly.graph_objects as go

from datetime import datetime, timedelta

import influxdb_client

from tastytrade.common.logging import setup_logging
from tastytrade.connections.sockets import DXLinkManager
from tastytrade.connections import Credentials, InfluxCredentials
from tastytrade.analytics.visualizations.charts import Study

from tastytrade.providers.market import MarketDataProvider

from tastytrade.analytics.indicators.momentum import hull

# Show all rows in pandas DataFrames
pl.Config.set_fmt_str_lengths(100)
pl.Config.set_tbl_rows(100)
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_width_chars(None)

logging.getLogger().handlers.clear()

TEST = True

setup_logging(
    level=logging.INFO,
    log_dir="../logs",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)

loop = asyncio.get_event_loop()
loop.set_debug(True)
logging.getLogger("asyncio").setLevel(logging.DEBUG)

## Test individual components

In [ ]:
# Set API credentials
credentials = Credentials(env="Live")
dxlink = DXLinkManager()

await dxlink.open(credentials=credentials)

In [3]:
influx_user = InfluxCredentials()
influxdb = influxdb_client.InfluxDBClient(
    url=InfluxCredentials().url, token=influx_user.token, org=influx_user.org
)

In [4]:
streamer = MarketDataProvider(dxlink, influxdb)

candles: pl.DataFrame = streamer.download(
    symbol="SPX{=5m}",
    event_type="CandleEvent",
    start=datetime(2025, 2, 21, 9) + timedelta(hours=5),
    stop=datetime(2025, 2, 21, 16) + timedelta(hours=5),
    debug_mode=True,
)

In [ ]:
chart_config = {"height": 800, "xaxis_rangeslider_visible": True}

fig = go.Figure(
    data=[
        go.Candlestick(
            x=candles["time"].dt.replace_time_zone("UTC").dt.convert_time_zone("America/New_York"),
            open=candles["open"],
            high=candles["high"],
            low=candles["low"],
            close=candles["close"],
        )
    ]
)
fig.update_layout(**chart_config)
fig.show()

In [ ]:
hull(input_df=candles)

In [9]:
hma_study = Study(
    name="HMA-20",
    compute_fn=hull,  # The hull function
    params={"length": 20},  # Parameters for hull function
    plot_params={
        "colors": {"Up": "#01FFFF", "Down": "#FF66FE"},  # Green for uptrend  # Red for downtrend
        "width": 2,
    },
    value_column="HMA",
    color_column="HMA_color",
)